In [12]:
import openai_api as opai
from create_hed_prompts import get_hed_vocab, get_coco_hed
import pandas as pd
import json
import re

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## From HED tags to description

In [13]:
messages=[
        {"role": "system", "content": "You are a helpful assistant. You convert Hierarchical Event Descriptor (HED) tags into plain English descriptions."},
        {"role": "system", "content": "HED: (Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))"},
        {"role": "system", "content": "Description: The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "user", "content": "HED: (Foreground-view, ((Item-count, High), Ingestible-object)), (Background-view, ((Human, Body, Agent-trait/Adult), Outdoors, Furnishing, Natural-feature/Sky, Urban, Man-made-object))"}
]
opai.prompt_engineer(messages,"gpt-4o-mini")

'Description: A scene depicting a high number of ingestible objects in the foreground, with the background showing an adult human body in an outdoor setting that includes natural features like the sky, as well as urban elements and man-made objects.'

In [18]:
messages=[
        {"role": "system", "content": "You are a good writer."},
        {"role": "user", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))?"},
        {"role": "assistant", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "user", "content": "(Foreground-view, ((Item-count, High), Ingestible-object)), (Background-view, ((Human, Body, Agent-trait/Adult), Outdoors, Furnishing, Natural-feature/Sky, Urban, Man-made-object))"}
]
opai.prompt_engineer(messages,"gpt-3.5-turbo")

'In the foreground view, there is a high count of ingestible objects, indicating a variety of food or drinks. In the background view, there is an adult human body, outdoors, with various furnishing and natural features such as the sky. Additionally, there are man-made objects characteristic of an urban environment.'

In [19]:
messages=[
        {"role": "system", "content": "You are a concise translator."},
        {"role": "user", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))?"},
        {"role": "assistant", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "user", "content": "(Foreground-view, ((Item-count, High), Ingestible-object)), (Background-view, ((Human, Body, Agent-trait/Adult), Outdoors, Furnishing, Natural-feature/Sky, Urban, Man-made-object))"}
]
opai.prompt_engineer(messages,"gpt-3.5-turbo")

'In the foreground, there is a high count of ingestible objects. In the background, there is an adult human body, outdoors, with a combination of natural features such as the sky and urban elements including man-made objects and furnishings.'

In [27]:
messages=[
        {"role": "system", "content": "You are a scientific writer."},
        {"role": "user", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))?"},
        {"role": "assistant", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "user", "content": "(Foreground-view, ((Item-count, High), Ingestible-object)), (Background-view, ((Human, Body, Agent-trait/Adult), Outdoors, Furnishing, Natural-feature/Sky, Urban, Man-made-object))"}
]
opai.prompt_engineer(messages,"gpt-3.5-turbo")

'In the foreground view, there is a high number of ingestible objects. \nIn the background view, there is an adult human body with agent traits, outdoors with furnishings, natural features such as the sky, and man-made objects in an urban setting.'

### Translate COCO HED tags

In [17]:
coco_hed = get_coco_hed()
coco_hed.head()

,File_Name,nsd_id,cocoId,HED_short,HED_long
0,shared0001_nsd02951.png,2950,262145,"(Foreground-view, ((Item-count, High), Ingesti...",(Property/Sensory-property/Sensory-presentatio...
1,shared0002_nsd02991.png,2990,262239,"(Foreground-view, ((Item-count/1, (Human, Body...",(Property/Sensory-property/Sensory-presentatio...
2,shared0003_nsd03050.png,3049,262414,"(Foreground-view, ((Item-count/1, ((Human, Hum...",(Property/Sensory-property/Sensory-presentatio...
3,shared0004_nsd03078.png,3077,524646,"(Foreground-view, (((Item-count/1, ((Human, Hu...",(Property/Sensory-property/Sensory-presentatio...
4,shared0005_nsd03147.png,3146,262690,"(Foreground-view, ((Item-count/1, Building), (...",(Property/Sensory-property/Sensory-presentatio...


In [52]:
messages=[
        {"role": "system", "content": "You are a scientific writer."},
        {"role": "user", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))?"},
        {"role": "assistant", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "user", "content": "<target_annotation>"}
]

translation = []
for idx, row in coco_hed.iterrows():
    messages[-1] = {"role": "user", "content": f"{row['HED_short']}"}
    output = opai.prompt_engineer(messages,"gpt-3.5-turbo")
    print(output)
    translation.append(output)
coco_hed_openai = coco_hed.copy()   
coco_hed_openai["openai_desc"] = translation
with open('coco_hed_openai', 'w') as fout:
    coco_hed_openai.to_csv(fout, index=False, sep='\t')

In the foreground view, there is a high number of ingestible objects. 

In the background view, there is a human with adult body characteristics, outdoors with various furnishings and natural features such as the sky, as well as urban and man-made objects.


KeyboardInterrupt: 

In [36]:
for idx, row in coco_hed.iterrows():
    if idx < len(translation):
        continue
    messages[-1] = {"role": "user", "content": f"{row['HED_short']}"}
    output = opai.prompt_engineer(messages,"gpt-3.5-turbo")
    print(output)
    translation.append(output)
coco_hed_openai = coco_hed.copy()   
coco_hed_openai["openai_desc"] = translation
with open('coco_hed_openai', 'w') as fout:
    coco_hed_openai.to_csv(fout, index=False, sep='\t')

In the foreground view, there is an animal and a piece of furnishing, each appearing once. In the background view, there is a plant outdoors near a building.
In the foreground view, there is one item which appears to be a furnishing. In the background view, there is a building in an urban setting outdoors, with a natural feature of the sky.
The foreground view consists of a single aircraft, while the background view is of the sky, depicting an outdoor natural feature.
In the foreground, there is an animal, and next to it, there are two pieces of furnishing. The scene takes place indoors.
In the foreground view, there is a composition of a human agent, a male adult body, and a man-made toy object. Alongside them are characters that are present in half of the total count. The foreground also includes a composite terrain. In the background view, there is an outdoor field with plants.
In the foreground view, we see a young male child with a human body. Nearby, there is an adult male figure

In [37]:
len(translation)

1000

In [41]:
filenames_downsampled = [f"{name.split('.')[0]}_downsampled.png" for name in coco_hed_openai["File_Name"]]
coco_hed_openai_downsampled = coco_hed_openai.copy()
coco_hed_openai_downsampled["File_Name_Downsampled"] = filenames_downsampled
with open('coco_hed_openai_downsampled', 'w') as fout:
    coco_hed_openai_downsampled.to_csv(fout, index=False, sep='\t')

In [53]:
coco_hed_web_data = coco_hed_openai_downsampled[['File_Name_Downsampled','HED_short', 'openai_desc']]

# Serializing json
json_object = json.dumps(coco_hed_web_data.to_dict(orient='list'), indent=4)
 
# Writing to sample.json
with open("slideshow_data.json", "w") as outfile:
    outfile.write(json_object)

## From description to HED tags

Retrieve HED terms from HEDLatest.xml on Github

In [4]:
hed_vocab = get_hed_vocab()
print(hed_vocab)

Event,Sensory-event,Agent-action,Data-feature,Experiment-control,Experiment-procedure,Experiment-structure,Measurement-event,Agent,Animal-agent,Avatar-agent,Controller-agent,Human-agent,Robotic-agent,Software-agent,Action,Communicate,Communicate-gesturally,Clap-hands,Clear-throat,Frown,Grimace,Nod-head,Pump-fist,Raise-eyebrows,Shake-fist,Shake-head,Shhh,Shrug,Smile,Spread-hands,Thumb-up,Thumbs-down,Wave,Widen-eyes,Wink,Communicate-musically,Hum,Play-instrument,Sing,Vocalize,Whistle,Communicate-vocally,Cry,Groan,Laugh,Scream,Shout,Sigh,Speak,Whisper,Move,Breathe,Blow,Cough,Exhale,Hiccup,Hold-breath,Inhale,Sneeze,Sniff,Move-body,Bend,Dance,Fall-down,Flex,Jerk,Lie-down,Recover-balance,Shudder,Sit-down,Sit-up,Stand-up,Stretch,Stumble,Turn,Move-body-part,Move-eyes,Blink,Close-eyes,Fixate,Inhibit-blinks,Open-eyes,Saccade,Squint,Stare,Move-face,Bite,Burp,Chew,Gurgle,Swallow,Gulp,Yawn,Move-head,Lift-head,Lower-head,Turn-head,Move-lower-extremity,Curl-toes,Hop,Jog,Jump,Kick,Pedal,Press-foot,Run

/Users/dtyoung/Documents/HED/HED-LLM/create_hed_prompts.py:89: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(xml_content, "lxml")


In [11]:
messages=[
        {"role": "system", "content": "You are a precise translator."},
        {"role": "system", "content": f"Translate event descriptions into HED annotations. Please respond using only the following words: {hed_vocab}. Give response using just this vocabulary and do not use any other words.Translate event descriptions into HED annotations. Please respond using only the following words: {hed_vocab}. Give response using just this vocabulary and do not use any other words."},
        {"role": "system", "content": "Description: The foreground view consists of a large number of ingestible objects, indicating a high quantity. The background view includes an adult human body, outdoors in a setting that includes furnishings, natural features such as the sky, and man-made objects in an urban environment."},
        {"role": "system", "content": "HED: (Foreground-view, ((Item-count, High), Ingestible-object)), (Background-view, ((Human, Body, Agent-trait/Adult), Outdoors, Furnishing, Natural-feature/Sky, Urban, Man-made-object))"},
        {"role": "user", "content": "Description: A visual presentation of an image of a green square."},
        # {"role": "assistant", "content": "(Foreground-view, ((Item-count/1, (Human, Body, Agent-trait/Adult)), (Item-count/1, (Human, Body, (Face, Away-from), Male, Agent-trait/Adult)), ((Item-count, High), Furnishing))), (Background-view, (Ingestible-object, Furnishing, Room, Indoors, Man-made-object, Assistive-device))"},
        # {"role": "user", "content": "In the foreground view, we see a male adolescent human, characterized by his body and traits as an agent. He is interacting with a man-made object, possibly engaged in play or using it in some way. In the background view, the setting is outdoors, specifically showing a natural feature such as an ocean."},
        # {"role": "assistant", "content": "(Foreground-view, ((Item-count/1, (Human, Body, Agent-trait/Adult)), (Item-count/1, (Human, Body, (Face, Away-from), Male, Agent-trait/Adult)), ((Item-count, High), Furnishing))), (Background-view, (Ingestible-object, Furnishing, Room, Indoors, Man-made-object, Assistive-device))"},
]
output = opai.prompt_engineer(messages,"gpt-4o-mini")

# ## TODO: try few shots
# print('ground truth: (Foreground-view, ((Item-count/1, ((Human, Human-agent), Body, Male, Agent-trait/Adolescent)), (Play, (Item-count/1, Man-made-object)))), (Background-view, (Outdoors, Natural-feature/Ocean))')
# print(output)
# messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": f"Given this list of tags: {hed_vocab}."},
#         {"role": "user", "content": f"And given this annotation: {output}."},
#         {"role": "user", "content": f"Highlight the tags that are from the list by captioning them with *"},
# ]
# opai.prompt_engineer(messages,"gpt-3.5-turbo")
print(output)

HED: (Visual-presentation, (Image, (Geometric-object, Square, Color, Green)))


In [21]:
messages=[
        {"role": "system", "content": f"You are a precise translator"},
        {"role": "user", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "assistant", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))"},
        {"role": "user", "content": "A visual presentation of an image of an open market full of people and piles of vegetables."}
]
output = opai.prompt_engineer(messages,"gpt-3.5-turbo")

'(Visual-presentation,(Background-view,Open-market-scene),(Foreground-view,(People),(Piles-of-vegetables)))'

## Vector database

In [63]:
import chromadb
from chromadb.utils import embedding_functions
verbose = False

In [4]:
chroma_client = chromadb.Client()

In [64]:
def from_hed(input, hed_db):
    '''
    Replace tags in the input with tags from hed pulling from the vector database
    '''
    _, starts, ends = extract_tags_locs(input)
    
    result = ""
    curIdx = 0
    while curIdx < len(input):
        curChar = input[curIdx]
        
        if curIdx in starts:
            charIdx = starts.index(curIdx)
            hed_tag = retrieve_hed(input[starts[charIdx]:ends[charIdx]], hed_db)
            if verbose:
                print("output tag", hed_tag)
            result += hed_tag
            curIdx = ends[charIdx]
        else:
            result += curChar
            curIdx += 1
            
    return result

def extract_tags_locs(input):
    starts = []
    ends = []
    extracted_tags = []
    for match in re.finditer(r'\b\w+(-\w+)*\b', input):
        extracted_tags.append(match.group())
        starts.append(match.start())
        ends.append(match.end())
    return extracted_tags, starts, ends
    
def query_hed_db(input, hed_db):
    return hed_db.query(
        query_texts=[input],
        n_results=2
    )
                
def retrieve_hed(tag, hed_db):
    eps = 1e-05
    
    results = query_hed_db(tag, hed_db)
    
    if verbose:
        print("input tag", tag)
        print("results", results)
        
    for idx, neigh in enumerate(results['documents'][0]):
        dist = results['distances'][0][idx]
        # print(dist, dist <= eps)
        if dist <= eps:
            return neigh
        if verbose:
            print()
    # no exact match       
    return f"({','.join(results['documents'][0])})"


In [ ]:
collection = chroma_client.create_collection(name="hed_schema")

In [12]:
hed_vocab = get_hed_vocab()
hed_vocab_list = hed_vocab.split(',')

/Users/dtyoung/Documents/python-venv/LLM/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [54]:
query = "A snowboarder in red and orange snow gear picks himself up off the snowy ground."
messages=[
        {"role": "system", "content": f"You are a precise translator"},
        {"role": "user", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "assistant", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))"},
        {"role": "user", "content": query}
]
target_string = opai.prompt_engineer(messages,"gpt-3.5-turbo")
print(target_string)

(Visual-presentation, (Background-view, Snow-covered ground), (Foreground-view, ((Snowboarder, Red, Orange), (Snow-gear)), (Action, Picks-himself-up))))


In [9]:
collection.add(
    documents=hed_vocab.split(','),
    ids=["id"+str(i) for i in range(len(hed_vocab_list))],
    # metadatas=[{"source": "my_source"}, {"source": "my_source"}],
)

/Users/dtyoung/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|█| 79.3M/79.3M


In [95]:
print(from_hed(target_string, collection))

(Visual-presentation,(Background-view,(Open-eyes,Street-view)),(Foreground-view,((Human,Crowd-sound)),((Tomato,Plum))))


In [13]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=opai.get_openaikey(),
                model_name="text-embedding-ada-002"
            )
openai_collection = chroma_client.get_or_create_collection(name="hed_schema_openai", embedding_function=openai_ef)
openai_collection.add(
    documents=hed_vocab.split(','),
    ids=["id"+str(i) for i in range(len(hed_vocab_list))],
    # metadatas=[{"source": "my_source"}, {"source": "my_source"}],
)

In [58]:
print(from_hed(target_string, openai_collection))

input tag Visual-presentation
results {'ids': [['id986', 'id975']], 'distances': [[0.0, 0.17377419769763947]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['Visual-presentation', 'Sensory-presentation']]}
output tag Visual-presentation
input tag Background-view
results {'ids': [['id989', 'id991']], 'distances': [[0.0, 0.10612648725509644]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['Background-view', 'Foreground-view']]}
output tag Background-view
input tag Snow-covered
results {'ids': [['id698', 'id700']], 'distances': [[0.20974697172641754, 0.28285619616508484]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['Leaf-covered-terrain', 'Paved-terrain']]}


output tag (Leaf-covered-terrain,Paved-terrain)
input tag ground
results {'ids': [['id693', 'id224']], 'distances': [[0.22138914465904236, 0.2672097086906433]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['Terrain', 'Plant']]}


output tag (Terrain,Plant)
in

In [86]:
# messages_template = [
#         {"role": "system", "content": f"You are a precise translator"},
#         {"role": "user", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
#         {"role": "assistant", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))"},
# ]
messages_template=[
        {"role": "system", "content": "You are a precise translator."},
        {"role": "system", "content": f"Translate these sentences into tags using only tags from the provided list: {hed_vocab}."},
        {"role": "user", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "assistant", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))"},
]

In [87]:
df = pd.read_csv("shared1000_captions.tsv", sep="\t")
df.head()

,File Name,nsd_id,cocoId,id,caption
0,shared0527_nsd40154.png,40153,390120,307845,A snowboarder in red and orange snow gear pick...
1,shared0527_nsd40154.png,40153,390120,309501,A person on some skis in the snow.
2,shared0527_nsd40154.png,40153,390120,310176,A person in an orange ski suit skiing down a s...
3,shared0527_nsd40154.png,40153,390120,313680,A person in red jacket skiing on a bumpy slope.
4,shared0527_nsd40154.png,40153,390120,315318,A man riding skis down the snow covered hillside.


In [88]:
files = []
captions = []
generated_tags = []
remapped_tags = []

count = 30
for index, row in df.iterrows():
    # print(row)
    if index % 5 == 0:
        files.append(row['File Name'])
        
        caption = row['caption']
        captions.append(caption)
        print("Caption:", caption)
        messages = messages_template.copy()
        messages.append({"role": "user", "content": caption})
        output = opai.prompt_engineer(messages,"gpt-3.5-turbo")
        remapped = from_hed(output, openai_collection)
        
        generated_tags.append(output)
        remapped_tags.append(remapped)
        print("Generated tags:", output)
        print("Remapped to HED:", remapped)
        print("-"*100)
        print()
    if index > 5*count:
        break
final_df = pd.DataFrame.from_dict(dict(zip(['file','captions','generated','remapped'], [files, captions, generated_tags, remapped_tags])))
with open('openai_remapped_system_prompt.csv', 'w') as fout:
    final_df.to_csv(fout, index=False)

Caption: A snowboarder in red and orange snow gear picks himself up off the snowy ground.
Generated tags: (Action,Move-body-part,(Lower-extremity,(Leg,Motion,Fall-down)),(Upper-extremity,(Arm,Motion,Lift)),(Agent,(Snowboarder,(Color,Red),(Color,Orange),(Clothing,(Snow-gear,Snowy-ground))))
Remapped to HED: (Action,Move-body-part,(Lower-extremity,((Foot,Hip),(Move,Animation),Fall-down)),(Upper-extremity,((Upper-arm,Hand),(Room,Paper),Lift)),(Agent,((Boat,Bicycle),((Room,Paper),Red),(Color,Orange),((Clothing,Hair),((Snow,Clothing),(Snow,Terrain)))))
----------------------------------------------------------------------------------------------------

Caption: A group of people on skis on a snow covered slope.
Generated tags: (Object,Geometric-object,Snow-covered-slope,(Group-of,(People,Ski)))
Remapped to HED: (Object,Geometric-object,(Sloped-terrain,Leaf-covered-terrain),(Group-of,((Property,Human),(Lift,Boat))))
----------------------------------------------------------------------------

In [89]:
json_object = json.dumps(dict(zip(['file','captions','generated','remapped'], [files, captions, generated_tags, remapped_tags])), indent=4)
 
# Writing to sample.json
with open("hed_translator_system_slideshow_data.json", "w") as outfile:
    outfile.write(json_object)

In [95]:
messages=[
        {"role": "system", "content": "You are a precise translator."},
        {"role": "system", "content": f"Translate these sentences into tags using only tags from the provided comma-separated list: {hed_vocab}."},
        {"role": "system", "content": f"Please use parentheses to group related tags together preserving their semantic relationship."},
        {"role": "user", "content": "The visual presentation of a white cross and a grayscale image of a face with hair in a black background."},
        {"role": "assistant", "content": "(Visual-presentation,(Background-view,Black),(Foreground-view,((Center-of,Computer-screen),(Cross,White)),(Grayscale,(Face,Hair,Image))))"},
        {"role": "user", "content": "A snowboarder in red and orange snow gear picks himself up off the snowy ground."},
]
output = opai.prompt_engineer(messages,"gpt-3.5-turbo")

In [96]:
remapped = from_hed(output, openai_collection)

In [97]:
output

'(Move-body-part,(Lower-extremity,Snowy-ground),(Recover-balance,(Move-body,Snowboarder)),(Clothing,((Snow-gear,Red),(Snow-gear,Orange))))'

In [98]:
remapped

'(Move-body-part,(Lower-extremity,(Grassy-terrain,Leaf-covered-terrain)),(Recover-balance,(Move-body,(Boat,Bicycle))),(Clothing,(((Snow,Clothing),Red),((Snow,Clothing),(Orange,Yellow)))))'